In [1]:
!pip install resampy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install np_utils

In [4]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from tqdm import tqdm
import resampy

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense , Activation , Dropout , SpatialDropout2D, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, ReLU, Flatten
from keras.optimizers import Adam

from keras import backend as keras_backend
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l2

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold

import seaborn as sns
import random
import math

from glob import glob
from itertools import cycle

sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [6]:
def create_dataset_df(csv_file):
    dataset_df = pd.read_csv(csv_file)
    filepaths = []
    for i, row in dataset_df.iterrows():
        filepaths.append(os.path.join('/content/drive/MyDrive/ThesisData/', 'File'+str(row['fold']), row['slice_file_name']))
    dataset_df['filepath'] = filepaths
    return dataset_df

In [7]:
df = create_dataset_df('/content/drive/MyDrive/ThesisData/ThesisLionMD.csv')

In [8]:
df.head()

,fs_id,slice_file_name,classID,fold,class,filepath
0,iroar_123,iroar_123.wav,1,1,IRoar,/content/drive/MyDrive/ThesisData/File1/iroar_...
1,iroar_308,iroar_308.wav,1,1,IRoar,/content/drive/MyDrive/ThesisData/File1/iroar_...
2,grunt_878,grunt_878.wav,2,2,Grunt,/content/drive/MyDrive/ThesisData/File2/grunt_...
3,roar_178,roar_178.wav,0,1,Roar,/content/drive/MyDrive/ThesisData/File1/roar_1...
4,roar_963,roar_963.wav,0,3,Roar,/content/drive/MyDrive/ThesisData/File3/roar_9...


In [9]:
columns = ['filepath', 'class']

In [10]:
filtered_df = df[columns]

In [11]:
filtered_df.head()

,filepath,class
0,/content/drive/MyDrive/ThesisData/File1/iroar_...,IRoar
1,/content/drive/MyDrive/ThesisData/File1/iroar_...,IRoar
2,/content/drive/MyDrive/ThesisData/File2/grunt_...,Grunt
3,/content/drive/MyDrive/ThesisData/File1/roar_1...,Roar
4,/content/drive/MyDrive/ThesisData/File3/roar_9...,Roar


In [12]:
df_sound = filtered_df[filtered_df["class"] == "Grunt"]

In [13]:
df_sound.head()

,filepath,class
2,/content/drive/MyDrive/ThesisData/File2/grunt_...,Grunt
9,/content/drive/MyDrive/ThesisData/File3/grunt_...,Grunt
18,/content/drive/MyDrive/ThesisData/File3/grunt_...,Grunt
21,/content/drive/MyDrive/ThesisData/File2/grunt_...,Grunt
27,/content/drive/MyDrive/ThesisData/File3/grunt_...,Grunt


In [14]:
len(df_sound)

1491

In [15]:
# Set the maximum audio duration (in seconds)
max_audio_duration = 3

# Set the segment length (in seconds)
segment_length = 0.181406

# Initialize the desired number of segments
desired_num_segments = int(max_audio_duration / segment_length)

# Create an empty DataFrame to store the results
df = pd.DataFrame(columns=['File', 'Segment_Number', 'Start_Time', 'End_Time', 'Energy'])

# Parameters for segmentation
sampling_rate = 22050  # Sampling rate in Hz

In [16]:
audio_files = np.array(df_sound['filepath'])

In [17]:
# Iterate through the audio files
for audio_file in audio_files:
    # Load the audio file using librosa
    audio, sr = librosa.load(audio_file)

    # Calculate the number of samples for the segment length
    segment_samples = int(segment_length * sr)

    # Pad the audio to the maximum duration of 3 seconds
    max_samples = int(max_audio_duration * sr)
    audio = np.pad(audio, (0, max_samples - len(audio)), mode='constant')

    # Calculate the hop length based on the segment length
    constant_hop_length = segment_samples

    # Calculate the energy of each segment
    energy = np.array([sum(abs(audio[i:i+constant_hop_length])) for i in range(0, len(audio), constant_hop_length)])

    # Convert the energy values to numeric dtype
    energy = pd.to_numeric(energy)

    # Calculate the start and end times of each segment
    num_segments = len(energy)
    segment_duration = len(audio) / num_segments / sr  # Actual duration of each segment

    # All segments have the same start and end times
    start_times = np.arange(num_segments) * segment_duration
    end_times = start_times + segment_duration

    # Create a DataFrame for the current audio file
    audio_df = pd.DataFrame({'File': [audio_file] * num_segments,
                             'Segment_Number': np.arange(num_segments),
                             'Start_Time': start_times,
                             'End_Time': end_times,
                             'Energy': energy})

    # Append the results to the main DataFrame
    df = pd.concat([df, audio_df], ignore_index=True)

In [20]:
# Group by audio file and find the top 3 segments with maximum energy
top_2_segments = df.groupby('File').apply(lambda x: x.nlargest(2, 'Energy')).reset_index(drop=True)

# Print the top 3 segments for each audio file
print("Top 5 Segments for Each Audio File with Maximum Energy:")
print(top_2_segments)

Top 5 Segments for Each Audio File with Maximum Energy:
                                                   File Segment_Number  \
0     /content/drive/MyDrive/ThesisData/File1/grunt_...              0   
1     /content/drive/MyDrive/ThesisData/File1/grunt_...              1   
2     /content/drive/MyDrive/ThesisData/File1/grunt_...              1   
3     /content/drive/MyDrive/ThesisData/File1/grunt_...              2   
4     /content/drive/MyDrive/ThesisData/File1/grunt_...              1   
...                                                 ...            ...   
2977  /content/drive/MyDrive/ThesisData/File3/grunt_...              0   
2978  /content/drive/MyDrive/ThesisData/File3/grunt_...              0   
2979  /content/drive/MyDrive/ThesisData/File3/grunt_...              1   
2980  /content/drive/MyDrive/ThesisData/File3/grunt_...              0   
2981  /content/drive/MyDrive/ThesisData/File3/grunt_...              1   

      Start_Time  End_Time       Energy  
0       0.000

In [21]:
top_2_segments.head(15)

,File,Segment_Number,Start_Time,End_Time,Energy
0,/content/drive/MyDrive/ThesisData/File1/grunt_...,0,0.000000,0.176471,2600.796923
1,/content/drive/MyDrive/ThesisData/File1/grunt_...,1,0.176471,0.352941,2147.333688
2,/content/drive/MyDrive/ThesisData/File1/grunt_...,1,0.176471,0.352941,2996.625545
3,/content/drive/MyDrive/ThesisData/File1/grunt_...,2,0.352941,0.529412,2183.837027
4,/content/drive/MyDrive/ThesisData/File1/grunt_...,1,0.176471,0.352941,2012.529582
5,/content/drive/MyDrive/ThesisData/File1/grunt_...,0,0.000000,0.176471,1699.656008
6,/content/drive/MyDrive/ThesisData/File1/grunt_...,1,0.176471,0.352941,2796.095943
7,/content/drive/MyDrive/ThesisData/File1/grunt_...,2,0.352941,0.529412,1481.676341
8,/content/drive/MyDrive/ThesisData/File1/grunt_...,2,0.352941,0.529412,2090.174323
9,/content/drive/MyDrive/ThesisData/File1/grunt_...,1,0.176471,0.352941,2009.637250


In [25]:
top_segment = df['Segment_Number'].value_counts()
print(top_segment)

0     1491
9     1491
15    1491
14    1491
13    1491
12    1491
11    1491
10    1491
8     1491
1     1491
7     1491
6     1491
5     1491
4     1491
3     1491
2     1491
16    1491
Name: Segment_Number, dtype: int64


In [23]:
# Using series value_counts()
top_segment = top_2_segments['Segment_Number'].value_counts()
print(top_segment)

1    1463
0    1301
2     181
3      19
4      11
5       6
6       1
Name: Segment_Number, dtype: int64


In [24]:
point_start = 0.176471*22050
point_end = 0.352941*22050
print(point_start, point_end)

3891.1855499999997 7782.34905
